In [ ]:
ROOT_DIR = '/content'

In [ ]:
!pip install ultralytics PyYAML patool

In [ ]:
# @title Check Dangerous Animals { display-mode: "form" }
zebra = False # @param {type:"boolean"}
lion = True # @param {type:"boolean"}
rhino = True # @param {type:"boolean"}
elephant = False # @param {type:"boolean"}
buffalo = False  # @param {type:"boolean"}
turtle = True  # @param {type:"boolean"}
ostrich = False  # @param {type:"boolean"}

# Download Dataset (Images and Labels)

In [ ]:
%%bash

file_url="https://drive.google.com/uc?export=download&id=1FrPo0bICEH8Xwuyfl2TgBRycyQ6UYUtb"
destination_path="/content/wildlife_detector.zip"

confirmation_page=$(curl -s -L "$file_url")

file_id=$(echo "$confirmation_page" | grep -oE "name=\"id\" value=\"[^\"]+" | sed 's/name="id" value="//')
file_confirm=$(echo "$confirmation_page" | grep -oE "name=\"confirm\" value=\"[^\"]+" | sed 's/name="confirm" value="//')
file_uuid=$(echo "$confirmation_page" | grep -oE "name=\"uuid\" value=\"[^\"]+" | sed 's/name="uuid" value="//')

download_url="https://drive.usercontent.google.com/download?id=$file_id&export=download&confirm=$file_confirm&uuid=$file_uuid"

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

In [ ]:
import patoolib

patoolib.extract_archive("/content/wildlife_detector.zip", outdir=ROOT_DIR)

# Train YOLO Model with Custom Animal Dataset

In [ ]:
import os

from ultralytics import YOLO


def train_model():
  # Load a model
  model = YOLO("yolov8n.pt")  # build a pre-trained model


  # Use the model
  results = model.train(data=os.path.join(ROOT_DIR, "wildlife_detector", "config.yaml"), epochs=5)  # train the model

# Predict Animal Labels from Animal Videos

In [ ]:
import os

from ultralytics import YOLO
import cv2

def predict_model(video_file='zebra.mp4'):
  VIDEOS_DIR = os.path.join(ROOT_DIR, "wildlife_detector", 'videos')

  video_path = os.path.join(VIDEOS_DIR, video_file)
  video_path_out = '{}_out.mp4'.format(video_path.split('.')[0])

  cap = cv2.VideoCapture(video_path)
  ret, frame = cap.read()
  H, W, _ = frame.shape
  out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

  model_path = os.path.join(ROOT_DIR, 'runs', 'detect', 'train', 'weights', 'best.pt')

  # Load a model
  model = YOLO(model_path)  # load a custom model

  threshold = 0.5

  while ret:

      results = model(frame)[0]

      for result in results.boxes.data.tolist():
          x1, y1, x2, y2, score, class_id = result

          if score > threshold:
              color = (0, 0, 255)
              if "NOT DANGEROUS" in results.names[int(class_id)].upper():
                color = (0, 255, 0)
              cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
              cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                          cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

      out.write(frame)
      ret, frame = cap.read()

  cap.release()
  out.release()
  cv2.destroyAllWindows()

# Pre-processed Sample Animal Videos

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/wildlife_detector/videos/turtles.mp4','rb').read()
zebra_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/elephant_zebra_ostrich.mp4','rb').read()
lion_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/rhino_zebra.mp4','rb').read()
rhino_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (zebra_data_url, lion_data_url, rhino_data_url))

# Detect and Classify Animals in Videos using Model

In [ ]:
import yaml
import shutil

def predict_animal_detector(buffalo, lion, zebra, rhino, elephant, turtle, ostrich):
  # Delete Previous Runs
  shutil.rmtree('/content/runs/', ignore_errors=True)

  # Update Config based on front-end input
  with open("/content/wildlife_detector/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg['names']['0'] = 'buffalo (Dangerous)' if buffalo else 'buffalo (Not Dangerous)'
    cfg['names']['1'] = 'elephant (Dangerous)' if elephant else 'elephant (Not Dangerous)'
    cfg['names']['2'] = 'rhino (Dangerous)' if rhino else 'rhino (Not Dangerous)'
    cfg['names']['3'] = 'zebra (Dangerous)' if zebra else 'zebra (Not Dangerous)'
    cfg['names']['4'] = 'lion (Dangerous)' if lion else 'lion (Not Dangerous)'
    cfg['names']['5'] = 'ostrich (Dangerous)' if ostrich else 'ostrich (Not Dangerous)'
    cfg['names']['6'] = 'turtle (Dangerous)' if turtle else 'turtle (Not Dangerous)'

  with open("/content/wildlife_detector/config.yaml", "w") as f:
    cfg = yaml.dump(
        cfg, stream=f, default_flow_style=False, sort_keys=False
    )

  # Train YOLO Model with user input
  train_model()

  # Take 3 sample videos and use model to detect and classify animals
  for i in ['turtles.mp4', 'elephant_zebra_ostrich.mp4', 'rhino_zebra.mp4']:
    predict_model(i)

  # Convert video format to support browser
  os.system("ffmpeg -i /content/wildlife_detector/videos/turtles_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/turtles_out_final.mp4")
  os.system("ffmpeg -i /content/wildlife_detector/videos/elephant_zebra_ostrich_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/elephant_zebra_ostrich_out_final.mp4")
  os.system("ffmpeg -i /content/wildlife_detector/videos/rhino_zebra_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/rhino_zebra_out_final.mp4")


predict_animal_detector(buffalo, lion, zebra, rhino, elephant, turtle, ostrich)

# Post-processing Sample Animal videos (Detection and Classification)

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/wildlife_detector/videos/turtles_out_final.mp4','rb').read()
zebra_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/elephant_zebra_ostrich_out_final.mp4','rb').read()
lion_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/rhino_zebra_out_final.mp4','rb').read()
rhino_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (zebra_data_url, lion_data_url, rhino_data_url))